In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import regex as re
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Dense, Embedding, Dropout, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords') 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/MyDrive/TextAnal2")

In [4]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2022-04-16 18:21:11--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2022-04-16 18:21:11--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2022-04-16 18:21:11--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [5]:
!unzip glove.twitter.27B

Archive:  glove.twitter.27B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of glove.twitter.27B or
        glove.twitter.27B.zip, and cannot find glove.twitter.27B.ZIP, period.


In [6]:
!unzip file_location

unzip:  cannot find or open file_location, file_location.zip or file_location.ZIP.


In [7]:
!ls

 glove.twitter.27B.zip	 glove.twitter.27B.zip.1  'IMDB Dataset.csv'


In [8]:
!pwd

/content/drive/MyDrive/TextAnal2


In [9]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df.loc[df.review.isna()]  # 0 NaN values

,review,sentiment


In [11]:
df.loc[0,'review']

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [12]:
df['review'] = df.review.str.replace(r'<br /><br />', ' ')  # replace <br /><br /> with a whitespace

In [13]:
stop_words = Counter(stopwords.words('english'))  # faster look-up if dictionary
lemmatizer = WordNetLemmatizer()

def review_cleaner(review, stop_words):
    review = re.sub(r"[^a-zA-Z]", " ", review)
    review = re.sub(r'\s+', ' ', review, flags=re.I)  # flags to ignore case
    reduced_review = [word for word in review.lower().split() if word not in stop_words and len(word) > 1]
    lem_review = [lemmatizer.lemmatize(word) for word in reduced_review if len(lemmatizer.lemmatize(word)) > 1]
    return ' '.join(lem_review)

In [14]:
df['review'] = df.review.apply(lambda x: review_cleaner(x, stop_words))

In [15]:
df.loc[0, 'review']

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz m

In [16]:
df.sentiment.replace({'positive': 1, 'negative': 0}, inplace=True)
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter mattei love time money visually stunnin...,1


In [17]:
XX_train, X_test, yy_train, y_test = train_test_split(df.review.tolist(), df.sentiment.tolist(), test_size=0.2, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(XX_train, yy_train, test_size=0.15, random_state=42)

#### --------------------------------------------------------Embeddings----------------------------------------------------------------------------------------------

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

v_size = len(tokenizer.word_index) + 1

In [20]:
train_sequences = tokenizer.texts_to_sequences(X_train)
dev_sequences = tokenizer.texts_to_sequences(X_dev)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [21]:
sent_lengths = [len(s.split()) for s in X_train]
mean_l_sent = np.mean(sent_lengths)
std_l_sent = np.std(sent_lengths)

max_len = round(mean_l_sent + std_l_sent)

In [22]:
train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating='post', padding='post')
dev_padded = pad_sequences(dev_sequences, maxlen=max_len, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

In [26]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.twitter.27B.200d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.


FileNotFoundError: ignored

In [27]:
embedding_matrix = np.zeros((v_size, 200))

for word, i in tokenizer.word_index.items():
    if i < v_size:
        emb_vec = embeddings_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

NameError: ignored

In [ ]:
embedding_matrix.shape

In [ ]:
model = Sequential()

model.add(Embedding(v_size, 200, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(learning_rate=0.0001)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics='accuracy')

model.summary()

In [ ]:
history = model.fit(train_padded, np.array(y_train), epochs=1000, batch_size=32,
                    validation_data=(dev_padded, np.array(y_dev)), callbacks=[es])

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
vectorizer = TfidfVectorizer()
X_vect_train = vectorizer.fit_transform(X_train)
X_vect_dev = vectorizer.transform(X_dev)
X_vect_test = vectorizer.transform(X_test)

In [25]:
def MyHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Input(shape=(32, 32, 3)))
        model_type = hp.Choice("model_type", ["mlp", "cnn"])
        if model_type == "mlp":
            with hp.conditional_scope("model_type", ["mlp"]):
                model.add(Flatten())
                model.add(Dense(32, activation='relu'))
        if model_type == "cnn":
            with hp.conditional_scope("model_type", ["cnn"]):
                model.add(Conv2D(64, 3, activation='relu'))
                model.add(GlobalAveragePooling2D())
        model.add(Dense(10, activation='softmax'))
        return model